In [ ]:
!pip install psycopg2

In [1]:
import pandas as pd
pd.options.display.max_rows = 10
hostname= 'localhost'
database= 'Semana6_DE'
username= 'postgres'
pwd='david9.25.38'
port_id= '5432'
import psycopg2

In [2]:
# Creamos la conexion (local)
conn = psycopg2.connect(host=hostname, dbname=database, user=username, password=pwd, port=5432)
# Conexion a redshift
#conn = psycopg2.connect(host=hostname, dbname=database, user=username, password=pwd, port=5439)

In [3]:
def execute_read_query(connection, query):
    cursor = connection.cursor()
    result = None
    try:
        cursor.execute(query)
        result = cursor.fetchall()
        return result
    except Error as e:
        print(f"Error '{e}' ha ocurrido")

**Pregunta 1**

Encuentre los cinco vendedores con mejor desempeño usando la columna `salesytd` (Sales, year-to-date). (Solo necesitamos conocer el `businessentityid` de cada vendedor, ya que esto identifica de forma única a cada uno). ¿Por qué podría ser escéptico con estos números en este momento?

In [4]:
query5="""SELECT BusinessEntityID, SalesYTD FROM SalesPerson ORDER BY SalesYTD DESC LIMIT 5;"""
execute_read_query(conn, query5)

[(276, 4251368.5497),
 (289, 4116871.2277),
 (275, 3763178.1787),
 (277, 3189418.3662),
 (290, 3121616.3202)]

Una version mejorada de la funcion sería

In [13]:
cursor = conn.cursor()
cursor.execute(query5)
columnas = [description[0] for description in cursor.description]
cursor.fetchall()
print(columnas)
execute_read_query(conn, query5)
pd.DataFrame(execute_read_query(conn, query5),columns=columnas)

['businessentityid', 'salesytd']


,businessentityid,salesytd
0,276,4.251369e+06
1,289,4.116871e+06
2,275,3.763178e+06
3,277,3.189418e+06
4,290,3.121616e+06


Los números están codificados en esta tabla, en lugar de calcularse dinámicamente a partir de cada registro de ventas. Actualmente, no sabemos cómo se actualiza este número o mucho al respecto, por lo que es bueno permanecer escéptico.

**Pregunta 2**

Usando ```salesorderheader```, busque los 5 mejores vendedores que hicieron la mayor cantidad de ventas **en el año más reciente** (2014). (Hay una columna llamada `subtotal`; úsela). Las ventas que no tienen un vendedor asociado deben excluirse de sus cálculos y producción final. Se deben incluir todos los pedidos que se realizaron dentro del año calendario 2014.

**Pista:** Puedes usar la sintaxis `'1970-01-01'` para generar un punto de comparacion en el tiempo

In [14]:
query6="""
SELECT salespersonid, round(SUM(subtotal)) AS totalsales
FROM salesorderheader soh
WHERE soh.orderdate >= '2014-01-01'
AND soh.SalesPersonID is not NULL
GROUP BY SalesPersonID
ORDER BY TotalSales DESC
LIMIT 5;
"""
execute_read_query(conn, query6)

[(289.0, 1382997.0),
 (276.0, 1271089.0),
 (275.0, 1057247.0),
 (282.0, 1044811.0),
 (277.0, 1040093.0)]

In [15]:
cursor = conn.cursor()
cursor.execute(query6)
columnas = [description[0] for description in cursor.description]
cursor.fetchall()
print(columnas)
pd.DataFrame(execute_read_query(conn, query6),columns=columnas)

['salespersonid', 'totalsales']


,salespersonid,totalsales
0,289.0,1382997.0
1,276.0,1271089.0
2,275.0,1057247.0
3,282.0,1044811.0
4,277.0,1040093.0
